In [13]:
import pandas as pd  
import numpy as np
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
import re
from gensim import utils
from gensim.models.doc2vec import LabeledSentence
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score
import multiprocessing
from tqdm import tqdm


In [2]:
csv = 'reviews_clean.csv'
my_df = pd.read_csv(csv,index_col=0)
my_df.head()

,text,stars
0,first admit not excite go la tavolta food snob...,4.0
1,walk around friday afternoon sit table bar wal...,1.0
2,not really find anything wrong place pastas pi...,5.0
3,great lunch today staff helpful assist selecti...,4.0
4,good selection class beers mains twice first t...,3.0


In [3]:
my_df.dropna(inplace=True)
my_df.reset_index(drop=True,inplace=True)
my_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 864530 entries, 0 to 864529
Data columns (total 2 columns):
text     864530 non-null object
stars    864530 non-null float64
dtypes: float64(1), object(1)
memory usage: 13.2+ MB


In [4]:
x = my_df.text
y = my_df.stars

In [5]:
tvec = TfidfVectorizer(max_features=5000)
tvec.fit(my_df['text'])
x_tfidf = tvec.transform(my_df['text'])
# print(tvec.vocabulary_)

In [6]:
nb_model = MultinomialNB()
nb_model.fit(x_tfidf, y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [56]:
nb_predictions = nb_model.predict(x_tfidf)
# print("Naive Bayes Accuracy Score -> ",accuracy_score(nb_predictions, y)*100)

In [8]:
def labelize_tweets_ug(tweets,label):
    result = []
    prefix = label
    for i, t in zip(tweets.index, tweets):
        result.append(LabeledSentence(t.split(), [prefix + '_%s' % i]))
    return result

all_x_w2v = labelize_tweets_ug(x, 'all')

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  """


In [14]:
cores = multiprocessing.cpu_count()
model_ug_dmc = Doc2Vec(dm=1, dm_concat=1, size=100, window=2, negative=5, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_ug_dmc.build_vocab([x for x in tqdm(all_x_w2v)])

100%|██████████| 864530/864530 [00:00<00:00, 2423670.32it/s]


In [15]:
for epoch in range(30):
    model_ug_dmc.train(shuffle([x for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)
    model_ug_dmc.alpha -= 0.002
    model_ug_dmc.min_alpha = model_ug_dmc.alpha

100%|██████████| 864530/864530 [00:00<00:00, 2984824.12it/s]


In [16]:
model_ug_dmc.save('d2v_model_ug_dmc.doc2vec')
model_ug_dmc = Doc2Vec.load('d2v_model_ug_dmc.doc2vec')

In [17]:
model_ug_dmc.most_similar('good')

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('great', 0.7834584712982178),
 ('awesome', 0.7669825553894043),
 ('amaze', 0.7517189979553223),
 ('terrific', 0.728622555732727),
 ('excellent', 0.7282618284225464),
 ('incredible', 0.7237319946289062),
 ('outstanding', 0.7165056467056274),
 ('fantastic', 0.7058401107788086),
 ('fabulous', 0.7038179039955139),
 ('delicious', 0.7003250122070312)]

In [18]:
model_ug_dmc.most_similar('truffle')

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('garlic', 0.7286983728408813),
 ('truffled', 0.7102000713348389),
 ('cajun', 0.6667969822883606),
 ('trufle', 0.655715823173523),
 ('garluc', 0.6231197118759155),
 ('telleggio', 0.6226471662521362),
 ('suicide', 0.6222496032714844),
 ('galric', 0.6217901706695557),
 ('french', 0.6211013197898865),
 ('asiago', 0.6209801435470581)]

In [22]:
model_ug_dmc.most_similar('noisy')

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('loud', 0.867179811000824),
 ('hectic', 0.7589107751846313),
 ('cramp', 0.7433356046676636),
 ('noisey', 0.7396436929702759),
 ('rowdy', 0.7214796543121338),
 ('chaotic', 0.6943398714065552),
 ('garrish', 0.6683323383331299),
 ('congest', 0.66129469871521),
 ('crowd', 0.6584668755531311),
 ('busy', 0.6558099985122681)]

In [23]:
model_ug_dmc.most_similar('bad')

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('terrible', 0.7330342531204224),
 ('horrible', 0.6986491680145264),
 ('aout', 0.6849514245986938),
 ('badd', 0.6782817840576172),
 ('poor', 0.655217170715332),
 ('awful', 0.6214576959609985),
 ('good', 0.6200417876243591),
 ('lousy', 0.6163746118545532),
 ('unmissable', 0.6094075441360474),
 ('shitty', 0.5988847017288208)]

In [21]:
model_ug_dmc.most_similar('avocado')

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('avacado', 0.8558058142662048),
 ('jalapenos', 0.7609074711799622),
 ('guacamole', 0.734514594078064),
 ('pineapple', 0.7290098667144775),
 ('prosciutto', 0.7194356918334961),
 ('bacon', 0.7095621824264526),
 ('cilantro', 0.6944547295570374),
 ('ham', 0.6935070157051086),
 ('avocados', 0.6919876337051392),
 ('cuc', 0.6896876692771912)]

In [24]:
model_ug_dmc.most_similar('toilet')

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('bathroom', 0.6078024506568909),
 ('toliet', 0.6061931848526001),
 ('tissue', 0.6043602228164673),
 ('alsp', 0.5717312693595886),
 ('parchment', 0.5563274621963501),
 ('soap', 0.5523649454116821),
 ('toilets', 0.5482754111289978),
 ('restroom', 0.5459871292114258),
 ('whitout', 0.5351422429084778),
 ('urinal', 0.5248912572860718)]

In [26]:
model_ug_dmc.most_similar('organic')

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('sustainable', 0.723642110824585),
 ('locally', 0.6479333639144897),
 ('source', 0.6410207748413086),
 ('halal', 0.6226398348808289),
 ('natural', 0.6219726204872131),
 ('groud', 0.6083687543869019),
 ('checkfresh', 0.590692400932312),
 ('gmo', 0.5891855359077454),
 ('beercat', 0.5880138874053955),
 ('local', 0.5876971483230591)]

In [44]:
model_ug_dmc.most_similar('cheap')

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('inexpensive', 0.8349385857582092),
 ('affordable', 0.7634541392326355),
 ('cheep', 0.7321719527244568),
 ('expensive', 0.6124028563499451),
 ('healthy', 0.6088963150978088),
 ('spendy', 0.5893974304199219),
 ('unhealthy', 0.5869796276092529),
 ('babely', 0.5741486549377441),
 ('reasonable', 0.5647486448287964),
 ('cheapest', 0.55983567237854)]

In [30]:
model_ug_dmc.most_similar('pancake')

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('pancakes', 0.7315992116928101),
 ('crepe', 0.7205942273139954),
 ('cheesecake', 0.6569241285324097),
 ('milkshake', 0.6226840019226074),
 ('torta', 0.5992645025253296),
 ('frenchtoast', 0.5989701747894287),
 ('waffle', 0.5944867134094238),
 ('portals', 0.5836019515991211),
 ('oommmggg', 0.5797628164291382),
 ('mousse', 0.5750806331634521)]

In [31]:
model_ug_dmc.most_similar('garlic')

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('garluc', 0.7472913265228271),
 ('truffle', 0.7286984324455261),
 ('garloc', 0.7043008804321289),
 ('gsrlic', 0.6788638830184937),
 ('starsgarlic', 0.673371434211731),
 ('galic', 0.667336106300354),
 ('cinnamon', 0.6671755313873291),
 ('cheezey', 0.6643325090408325),
 ('garlc', 0.6634542346000671),
 ('herb', 0.6601952314376831)]

In [34]:
model_ug_dmc.most_similar(['staff', 'rude'])

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('waitstaff', 0.7390881776809692),
 ('arguementitive', 0.7033827304840088),
 ('unprofessional', 0.6949495077133179),
 ('sedum', 0.6853319406509399),
 ('effcient', 0.678861677646637),
 ('competant', 0.6784259080886841),
 ('bartenders', 0.6777243614196777),
 ('insolent', 0.6776140928268433),
 ('kiela', 0.676659882068634),
 ('knowelgable', 0.6706382036209106)]

In [35]:
model_ug_dmc.most_similar('wait')

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('waut', 0.6780174970626831),
 ('chillidog', 0.6552867889404297),
 ('ittook', 0.6449674367904663),
 ('diaster', 0.6426888704299927),
 ('fastforward', 0.636142373085022),
 ('dissapears', 0.6352733373641968),
 ('impasse', 0.6351890563964844),
 ('waite', 0.6343010663986206),
 ('desapointed', 0.6252535581588745),
 ('tookas', 0.6134780049324036)]

In [36]:
model_ug_dmc.most_similar(['burger', 'impossible'])

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('pizza', 0.6852853298187256),
 ('hamburger', 0.655759334564209),
 ('surrealist', 0.6153266429901123),
 ('xgx', 0.6147144436836243),
 ('toplobster', 0.6052716970443726),
 ('alferdo', 0.6028608679771423),
 ('kdrbbig', 0.6025829911231995),
 ('tume', 0.6008009910583496),
 ('oommmggg', 0.5991754531860352),
 ('usmenuguide', 0.5977445840835571)]

In [37]:
model_ug_dmc.most_similar('sparkle')

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('bottle', 0.6409244537353516),
 ('sprits', 0.6106453537940979),
 ('withwhite', 0.6044120788574219),
 ('alkaline', 0.594031810760498),
 ('pitcher', 0.5862404108047485),
 ('badoit', 0.5823096036911011),
 ('hinterland', 0.5799775123596191),
 ('kirin', 0.5765917301177979),
 ('coors', 0.5752911567687988),
 ('embalm', 0.5736021995544434)]

In [38]:
model_ug_dmc.most_similar(['root', 'beer'])

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('draft', 0.7703471183776855),
 ('wine', 0.6967378258705139),
 ('hefeweisen', 0.6879545450210571),
 ('peroni', 0.6850576400756836),
 ('brouwerij', 0.6842464208602905),
 ('draught', 0.6729234457015991),
 ('rootbeer', 0.6687580943107605),
 ('soda', 0.6608607172966003),
 ('microbrasserie', 0.660210371017456),
 ('pnls', 0.6578246355056763)]

In [39]:
model_ug_dmc.most_similar('rootbeer')

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('coke', 0.6155045032501221),
 ('guiness', 0.6096252799034119),
 ('cheerwine', 0.6074126958847046),
 ('watm', 0.6037907600402832),
 ('sundrop', 0.592220664024353),
 ('soda', 0.590106189250946),
 ('margaritas', 0.5832696557044983),
 ('terredora', 0.5822097659111023),
 ('rootbear', 0.5777513980865479),
 ('orangesicle', 0.5754582285881042)]

In [42]:
model_ug_dmc.most_similar('bleu')

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('blue', 0.8569976687431335),
 ('goat', 0.7580530643463135),
 ('pimento', 0.7245334386825562),
 ('fontina', 0.7224986553192139),
 ('feta', 0.7054100036621094),
 ('mac', 0.6765233278274536),
 ('blu', 0.6690124869346619),
 ('anatolian', 0.6682217121124268),
 ('swiss', 0.6648344993591309),
 ('gorgonzola', 0.66180020570755)]

In [43]:
model_ug_dmc.most_similar('smoothie')

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('milkshake', 0.7045836448669434),
 ('mojito', 0.6726048588752747),
 ('lemonade', 0.6669729948043823),
 ('sundae', 0.6608996391296387),
 ('bingsu', 0.6474591493606567),
 ('mimosa', 0.6382846832275391),
 ('poptart', 0.636228084564209),
 ('shiva', 0.6299512386322021),
 ('lasse', 0.6298040747642517),
 ('sorbet', 0.6214242577552795)]

In [54]:
model_ug_dmc.most_similar('overcook')

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('undercooked', 0.8836504817008972),
 ('overdo', 0.8363276720046997),
 ('underdone', 0.8045825958251953),
 ('rubbery', 0.7327951192855835),
 ('dry', 0.7217879891395569),
 ('mushy', 0.7167834043502808),
 ('underseasoned', 0.7003611922264099),
 ('bland', 0.6992430090904236),
 ('oily', 0.6963263154029846),
 ('oversauced', 0.694000244140625)]

In [55]:
model_ug_dmc.most_similar('breakfast')

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('brunch', 0.8187921643257141),
 ('bfast', 0.7082738876342773),
 ('sushi', 0.6280872821807861),
 ('nepolitan', 0.6137786507606506),
 ('greatbreakfast', 0.6060608625411987),
 ('carnisata', 0.5845016241073608),
 ('brekky', 0.5796356797218323),
 ('dinner', 0.5726624727249146),
 ('morrito', 0.5654523372650146),
 ('satisfier', 0.5650569200515747)]

In [67]:
def findSpeed(length, speed, pos):
    for num in pos:
        speed[num] += 10
    return sum(length[i]/speed[i] for i in range(len(speed)))
 

3.5

In [69]:
findSpeed([1000,1000,99,99],[50,50,49,49],[2,3])

43.35593220338983

In [70]:
findSpeed([1000,1000,99,99],[50,50,49,49],[0,1])

37.37414965986395